In [1]:
import os

In [2]:
%pwd

'c:\\Users\\manob\\Documents\\Kidney_Disease_Detection\\Kidney_Disease_Detection\\Pretrained Models\\MobileNetV3'

In [3]:
os.chdir("../")
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\manob\\Documents\\Kidney_Disease_Detection\\Kidney_Disease_Detection'

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt

c:\Users\manob\Documents\Kidney_Disease_Detection\Kidney_Disease_Detection\.conda\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\manob\Documents\Kidney_Disease_Detection\Kidney_Disease_Detection\.conda\Lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\manob\Documents\Kidney_Disease_Detection\Kidney_Disease_Detection\.conda\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [7]:
# Config
BASE_DIR = "artifacts/data_ingestion/Combined_Training_Data"
MODEL_PATH = "Pretrained Models/MobileNetV3/mobilenetv3_model.h5"
SAVE_BEST_MODEL_PATH = "Pretrained Models/MobileNetV3/mobilenetv3_model_updated.h5"

IMAGE_SIZE = (256, 256)
BATCH_SIZE = 32
EPOCHS = 40

In [8]:
# Data generators
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = datagen.flow_from_directory(
    BASE_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    BASE_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    shuffle=True
)


Found 31058 images belonging to 2 classes.
Found 7763 images belonging to 2 classes.


In [9]:
# Load model
model = load_model(MODEL_PATH)

# Callbacks
checkpoint_cb = ModelCheckpoint(SAVE_BEST_MODEL_PATH, save_best_only=True, monitor='val_loss', mode='min')
earlystop_cb = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [11]:
# Train
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[checkpoint_cb, earlystop_cb]
)


Epoch 1/40
971/971 [==============================] - 596s 614ms/step - loss: 0.6948 - accuracy: 0.5239 - val_loss: 0.7013 - val_accuracy: 0.5299
Epoch 2/40
971/971 [==============================] - 590s 608ms/step - loss: 0.6906 - accuracy: 0.5341 - val_loss: 0.7012 - val_accuracy: 0.3821
Epoch 3/40
971/971 [==============================] - 591s 609ms/step - loss: 0.6881 - accuracy: 0.5418 - val_loss: 0.7032 - val_accuracy: 0.4498
Epoch 4/40
971/971 [==============================] - 593s 611ms/step - loss: 0.6873 - accuracy: 0.5437 - val_loss: 0.7069 - val_accuracy: 0.5120
Epoch 5/40
971/971 [==============================] - 594s 612ms/step - loss: 0.6855 - accuracy: 0.5525 - val_loss: 0.7124 - val_accuracy: 0.4921


In [ ]:
# Plot
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title('Accuracy')
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss')
plt.show()
